In [33]:
import pandas as pd
import geopandas as gpd
import folium
from folium.plugins import HeatMap
from folium.plugins import Draw

# Load data
lahore = gpd.read_file(r"C:\Users\drfak\Toolkit\Lahore towns uc and vill\LH_TOWNS-2006_GEO_AREAS.shp")
df = pd.read_csv("ABCD.csv")

# Convert Date column to datetime object
df['Date'] = pd.to_datetime(df['Date'], format='%m/%d/%Y')

# Format Date column as string with same format
df['Date'] = df['Date'].dt.strftime('%Y-%m-%d')

# Convert Date column back to datetime object
df['Date'] = pd.to_datetime(df['Date'], format='%Y-%m-%d')

# Assuming 'Date' is the name of the datetime column
df.set_index('Date', inplace=True)

df

,Age,Sex,1st complaint,2nd Complaint,3rd Complaint,Address,Latitude,Longitude
Date,,,,,,,,
2011-05-01,18,2,fever,bodyaches,Headache,KOT ABDUL MALIK,31.620420,74.234381
2011-05-01,20,2,pain,bleeding,RTA,Gulshan-e-Ravi,31.552170,74.275290
2011-05-01,40,2,Dyspnea,chest pain,RTI,Gawalmandi,31.571870,74.318260
2011-05-01,24,2,headache,allergy,Vomiting,KOT ABDUL MALIK,31.620420,74.234381
2011-05-01,20,2,pain,bleeding,RTA,Gawalmandi,31.571870,74.318260
...,...,...,...,...,...,...,...,...
2011-08-31,31,2,Fever,Chills,Body pains,Gujranwala,32.166351,74.195900
2011-08-31,35,1,Vomiting,fever,nausea,Anarkali Bazaar Lahore,31.569800,74.312000
2011-08-31,35,1,Arthritis,Anxiety,arthralgia,Gawalmandi,31.571870,74.318260


In [83]:
dataset = gpd.GeoDataFrame(dataset, geometry=gpd.points_from_xy(dataset.Longitude, dataset.Latitude))

# Clip data to Lahore boundary
dataset = gpd.clip(dataset, lahore)

# Create KDE plot
kde = folium.plugins.HeatMap(dataset[['Latitude', 'Longitude']], 
                             name='KDE', 
                             radius=15, 
                             blur=25)

# Create map centered on Lahore
m = folium.Map(location=[31.582045, 74.329376], zoom_start=12)

# Add KDE plot to map
kde.add_to(m)

# Add Lahore regions shapefile to map
folium.GeoJson(lahore).add_to(m)

# Add layer control
folium.LayerControl().add_to(m)

# Add draw plugin to map
draw = Draw(export=True)
draw.add_to(m)


# Save map as HTML file
m.save('kde_map.html')

            Date  Age  Sex    1st complaint 2nd Complaint 3rd Complaint  \
64450 2011-08-20   32    1       High fever        Chills     Bodyaches   
68673 2011-08-25   25    2           chills         fever        chills   
74081 2011-08-31   70    1           chills      headache         fever   
61628 2011-08-15   22    2         tinnitis         fever           flu   
67865 2011-08-24   70    1         Vomiting  Running nose          Itch   
...          ...  ...  ...              ...           ...           ...   
63867 2011-08-18   35    2         pain abd      vomiting     bodyaches   
63869 2011-08-18   20    2            fever   muscle pain    joint pain   
63870 2011-08-18   23    1         Vomiting        Nausea         Fever   
63871 2011-08-18   22    2            fever     Bodyaches        chills   
63872 2011-08-18   58    1  Gastroenteritis        chills     bodyaches   

           Address   Latitude  Longitude                   geometry  
64450  Baghbanpura  31.584991

C:\Users\drfak\AppData\Local\Temp\ipykernel_15456\2904169211.py:5: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: None
Right CRS: GEOGCS["Geographic Coordinate System",DATUM["WGS84 ...

  dataset = gpd.clip(dataset, lahore)


In [2]:
from PyQt5.QtCore import QUrl
from PyQt5.QtWidgets import QApplication, QMainWindow, QWidget, QVBoxLayout, QPushButton
from PyQt5.QtWebEngineWidgets import QWebEngineView
from PyQt5.QtWidgets import QApplication, QMainWindow,QVBoxLayout,QAction,QFileDialog, QWidget, QSlider, QMessageBox, QTableWidget, QTableWidgetItem, QSizePolicy

import os
import sys
import json


class MainWindow(QMainWindow):
    def __init__(self):
        super().__init__()
        self.setWindowTitle("Interactive Map")
        central_widget = QWidget()
        self.setCentralWidget(central_widget)

        # Add a web engine view widget to the central widget
        layout = QVBoxLayout(central_widget)
        self.webview = QWebEngineView()
        layout.addWidget(self.webview)

        # Load the HTML file generated by Folium
        self.html_path = os.path.abspath('kde_map.html')

        # Connect to the downloadRequested signal
        self.webview.page().profile().downloadRequested.connect(self.handle_download_requested)

        # Add a push button to clear the canvas
        clear_button = QPushButton('Clear Map', self)
        clear_button.clicked.connect(self.clear_map)
        layout.addWidget(clear_button)

        # Add a push button to reload the HTML file
        reload_button = QPushButton('Reload Map', self)
        reload_button.clicked.connect(self.reload_map)
        layout.addWidget(reload_button)

    def handle_download_requested(self, item):
        # Get the suggested file name from the download item
        suggested_filename = item.suggestedFileName()
        if suggested_filename:
            # Open a file dialog to choose the save path
            save_path, _ = QFileDialog.getSaveFileName(
                self, 'Save File', suggested_filename
            )
            if save_path:
                # Set the save path for the download item
                item.setPath(save_path)
                item.accept()
                
                # Read the contents of the saved file and print them
                with open(save_path, 'r') as f:
                    data = json.load(f)
                # Get the coordinates of the geometry
                geometry_type = data['features'][0]['geometry']['type']
                if geometry_type == 'Polygon':
                    coordinates = data['features'][0]['geometry']['coordinates'][0]
                elif geometry_type == 'MultiPolygon':
                    coordinates = []
                    for polygon in data['features'][0]['geometry']['coordinates']:
                        coordinates.extend(polygon[0])
                else:
                    raise ValueError('Unsupported geometry type')

                # Initialize the minimum and maximum latitude and longitude values
                self.min_lat, self.max_lat = coordinates[0][1], coordinates[0][1]
                self.min_lon, self.max_lon = coordinates[0][0], coordinates[0][0]

                # Iterate over the coordinates and update the minimum and maximum values
                for coord in coordinates:
                    lon, lat = coord
                    if lat < self.min_lat:
                        self.min_lat = lat
                    elif lat > self.max_lat:
                        self.max_lat = lat
                    if lon < self.min_lon:
                        self.min_lon = lon
                    elif lon > self.max_lon:
                        self.max_lon = lon

                # Print the results
                print("Minimum latitude: ", self.min_lat)
                print("Maximum latitude: ", self.max_lat)
                print("Minimum longitude: ", self.min_lon)
                print("Maximum longitude: ", self.max_lon)


    def clear_map(self):
        # Clear the canvas by reloading a blank page
        self.webview.load(QUrl('about:blank'))

    def reload_map(self):
        # Reload the Folium-generated HTML file
        self.webview.load(QUrl.fromLocalFile(self.html_path))


if __name__ == '__main__':
    # Create the application and show the main window
    app = QApplication(sys.argv)
    main_window = MainWindow()
    main_window.show()

    # Start the event loop
    sys.exit(app.exec_())


SystemExit: 0

C:\Users\drfak\anaconda3\envs\geo_env\lib\site-packages\IPython\core\interactiveshell.py:3386: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
